In [1]:
import sys
sys.path.append("../") 

2 Model Building and Training

1 Split data using stratified train-test split

In [4]:
from src.data_preprocessor import DataPreprocessor
import pandas as pd

df = pd.read_csv("../data/processed/transformed_fraud_cleaned.csv")   
target_col = "class"                 

preprocessor = DataPreprocessor(df, target_col)

X_train, X_test, y_train, y_test = preprocessor.split()

print(X_train.shape, X_test.shape)


KeyError: "['class'] not found in axis"

2 BaselineModel (Logistic Regression)

In [3]:
from src.baseline_model import BaselineModel

baseline = BaselineModel()

# Train
baseline.train(X_train, y_train)

# Evaluate
baseline_results = baseline.evaluate(X_test, y_test)

print("Baseline Results:", baseline_results)


ValueError: could not convert string to float: '2015-07-30 15:35:29'